<a href="https://colab.research.google.com/github/Shubhranshu-Malhotra/Tensorflow-Developer-ZTM/blob/main/%5BPractice%5D_08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Basics in TensorFlow


## Get helper functions


In [1]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-05-31 10:44:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-31 10:44:19 (64.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Import helper functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Download a text dataset

We'll be using the [Real or Not?](https://www.kaggle.com/c/nlp-getting-started/data) datset from Kaggle which contains text-based Tweets about natural disasters. 


In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-05-31 10:47:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 172.253.123.128, 142.250.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-31 10:47:52 (103 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



`nlp_getting_started.zip` contains the following 3 `.csv` files:
* `sample_submission.csv` - an example of the file you'd submit to the Kaggle competition of your model's predictions.
* `train.csv` - training samples of real and not real diaster Tweets.
* `test.csv` - testing samples of real and not real diaster Tweets.

## Visualize text dataset

In [4]:
# Read the data
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)

In [9]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
# Visualize the test data
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
# How many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [15]:
# Total samples
len(train_df), len(test_df)

(7613, 3263)

In [18]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [30]:
# Visualize some random training sample
import random
for i in range(10):
  random_index = random.choice(range(0,len(train_df_shuffled)))
  target = train_df.iloc[random_index].target
  text = train_df.iloc[random_index].text
  print(f"Index: {random_index}")
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Index: 6077
Target: 1 (real disaster)
Text:
Sinkhole on west side damaging cars via @WEWS http://t.co/S7grbZNwlr

---

Index: 645
Target: 0 (not real disaster)
Text:
@Shayoly yes I love it

---

Index: 2523
Target: 0 (not real disaster)
Text:
yeah 1:57  |  Lamb Of God - Rock Am Ring 2015 - Intro+Desolation HD https://t.co/lEM5Z1NFk3 via @YouTube

---

Index: 7363
Target: 1 (real disaster)
Text:
IJ: Texas Seeks Comment on Rules for Changes to Windstorm Insurer http://t.co/h132iuL7MU

---

Index: 4296
Target: 0 (not real disaster)
Text:
@JYHeffect my good you stay in NY??? ?

---

Index: 6689
Target: 0 (not real disaster)
Text:
THIS IS RELAXING! #thunder #SoothMySlumber #WATERMELOANN #populardemand w/ @Soak... (Vine by @thewebbeffect19) https://t.co/F0QIRS5lJA

---

Index: 7490
Target: 0 (not real disaster)
Text:
GOP debate drinking game. For anyone looking for a bit of fun while watching this train wreck. http://t.co/W3Rga0nkOm http://t.co/0TZsQe8ESD

---

Index: 3605
Target: 0 (not rea

In [20]:
len(train_df_shuffled)

7613

### Split data into training and validation sets

Since the test set has no labels and we need a way to evalaute our trained models, we'll split off some of the training data and create a validation set.


In [32]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                  train_df_shuffled['target'].to_numpy(),
                                                  test_size = 0.1,
                                                  random_state = 42)

In [36]:
len(X_train), len(y_train), len(X_val), len(y_val)

(6851, 6851, 762, 762)

In [37]:
len(train_df_shuffled)

7613

In [38]:
len(X_train) + len(X_val)

7613

In [41]:
# Check the first 10 samples
X_train[:10], y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [42]:
X_val[:10], y_val[:10]

(array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
        'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
        'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
        '@camilacabello97 Internally and externally screaming',
        'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
        'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
        'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
        "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over so

## Converting text into numbers


### Method 1: Text Vecorization (Tokenization) 

In [45]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters [FOR LEARNING PURPOSE]
# Automaticalls adds <OOV> token for unknown words
test_vectorizer = TextVectorization(max_tokens=None, # num of words in vocabulary
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True
                                    )


In [48]:
len(X_train[0].split())

7

In [51]:
# Find the average number of tokes (word) in the training tweets
AVERAGE_LEN = round(sum([len(i.split()) for i in X_train])/len(X_train))
AVERAGE_LEN

15

In [52]:
# Set up text vectoriation variables
MAX_VOCAB_LEN = 10000
MAX_LEN = AVERAGE_LEN

In [55]:
text_vectorizer = TextVectorization(max_tokens= MAX_VOCAB_LEN,
                                    output_mode = 'int',
                                    output_sequence_length = MAX_LEN)

In [56]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(X_train)

In [57]:
type(X_train)

numpy.ndarray

In [70]:
# Create a sample sentence and tokenize it
sample_sentences = ['We had a 6.5 earthquake yesterday.', 'We will go for a picnic tomorrow.']
text_vectorizer(sample_sentences)

<tf.Tensor: shape=(2, 15), dtype=int64, numpy=
array([[  46,   94,    3,    1,  290, 1730,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  46,   38,  112,   10,    3,    1,  665,    0,    0,    0,    0,
           0,    0,    0,    0]])>

1 represents the \<OOV> token

In [71]:
# Choose a random sentence from training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f'Original text:\n {random_sentence},\
      \n \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 August 5 1620 one hundred-odd pilgrims from England and Holland set sail for the New World. They were unimpressed. http://t.co/pW5DSt9ROz,      
 
 Vectorized version:
 [[ 407  180 4185   61    1 9776   20 2126    7 2953  284 4694   10    2
    50]]


In [73]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in the vocab: {len(words_in_vocab)}")
print(f"5 most common words in vocab: {top_5_words}")
print(f"5 least common words in vocab: {bottom_5_words}")

Number of words in the vocab: 10000
5 most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
5 least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

The main parameters we're concerned about here are:
* `input_dim` - The size of the vocabulary (e.g. `len(text_vectorizer.get_vocabulary()`).
* `output_dim` - The size of the output embedding vector, for example, a value of `100` outputs a  feature vector of size 100 for each word.
* `embeddings_initializer` - How to initialize the embeddings matrix, default is `"uniform"` which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.
* `input_length` - Length of sequences being passed to embedding layer.


In [82]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = MAX_VOCAB_LEN,
                             output_dim = 128,
                             input_length = MAX_LEN)
embedding

In [83]:
# Get a random sentence from training set and pass through embedding layer
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence])) 
# Right now the representation is random but Will be trained during training to represent sentences better
sample_embed

Original text:
Pizza and beer in a tornado in Austin. Windy af right now      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02710241, -0.03940117,  0.03322748, ...,  0.0138473 ,
          0.03265047,  0.01615313],
        [-0.01768593, -0.03693328,  0.04749235, ...,  0.00018506,
          0.04619315,  0.01932073],
        [ 0.02174259,  0.02616348, -0.0263305 , ...,  0.00875334,
          0.01121379, -0.01064295],
        ...,
        [-0.00860177,  0.002195  ,  0.03046748, ...,  0.03435448,
          0.03927282,  0.0424802 ],
        [-0.00860177,  0.002195  ,  0.03046748, ...,  0.03435448,
          0.03927282,  0.0424802 ],
        [-0.00860177,  0.002195  ,  0.03046748, ...,  0.03435448,
          0.03927282,  0.0424802 ]]], dtype=float32)>

In [84]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02710241, -0.03940117,  0.03322748, -0.00197525,  0.00418018,
         0.04109621, -0.02172433,  0.01911299,  0.0421909 ,  0.00292374,
         0.04406803,  0.00344727,  0.02057833, -0.03448272,  0.00714367,
         0.00817432, -0.0196963 ,  0.01053535,  0.01997207, -0.00708705,
        -0.04668406, -0.03231076, -0.03317335, -0.04334083,  0.00664838,
         0.0403544 , -0.01658643,  0.02587331,  0.01126189,  0.02330388,
        -0.02133   ,  0.04508654,  0.00030184,  0.00131034, -0.04789979,
        -0.03526946, -0.02147988,  0.03097541,  0.01002539, -0.04063845,
        -0.04782208,  0.02623738, -0.02821236, -0.0155818 , -0.01514643,
         0.00480328,  0.03072102, -0.0137635 ,  0.04529699,  0.03947565,
        -0.03592378,  0.02366508, -0.01812452,  0.02854873,  0.04662044,
        -0.00254776, -0.04935285,  0.02769354,  0.03076423, -0.01861133,
         0.00797483, -0.00886854, -0.0141388 ,  0.04940102, -0.02850964,
  

## Modelling a text dataset (running a series of experiments)
we'll be building the following:
* **Model 0**: Naive Bayes (baseline) 
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: TensorFlow Hub Pretrained Feature Extractor
* **Model 7**: Same as model 6 with 10% of training data


Each experiment will go through the following steps:
* Construct the model
* Train the model
* Make predictions with the model
* Track prediction evaluation metrics for later comparison


### Model 0: Getting a baseline [Naive Bayes]

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.  
  
Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.  


To create our baseline, we'll create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert our words to numbers and then model them with the [Multinomial Naive Bayes algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB). This was chosen via referring to the [Scikit-Learn machine learning map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).


In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [89]:
# Evaluate our baseline model
baseline_score = model_0.score(X_val, y_val)
print(f"Our baseline model achieves the accuracy of: {baseline_score*100:0.2f}%")

Our baseline model achieves the accuracy of: 79.27%


In [90]:
# Make predictions 
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

* Accuracy
* Precision
* Recall
* F1-score


In [102]:
# add to helper functions
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
def get_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """

  metrics = {}
  metrics['accuracy'] = 100 * accuracy_score(y_true, y_pred)
  # Calculate model precision, recall and f1 score using "weighted" average
  metrics['precision'], metrics['recall'], metrics['f1_score'], _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  return metrics

In [100]:
baseline_results = get_results(y_val, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1_score': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}